In [1]:
using SparseArrays

In [2]:
include("../../src/ocean_model.jl")
include("../../src/schemes/helmholtz_sparse.jl")

sp_solve_modified_helmholtz

In [3]:
A = reshape(1:9, 3, 3)

for j in 1:3
    for i in 1:3
        println(A[i, j])
    end
end

1
2
3
4
5
6
7
8
9


In [4]:
# H_1 = 1.0
# H_2 = 2.0
# M = 4
# P = 4
# zeta = zeros(Float64, M+2, P+2, 2, 3)
# psi = zeros(Float64, M+2, P+2, 2, 3)

# # Initialise the model.
# for z in 1:2
#     a = rand(Float64, (M+2, P+2))
#     zeta[:,:,z,1] = update_ghost_cells(a)
#     b = rand(Float64, (M+2, P+2))
#     psi[:,:,z,1] = update_ghost_cells(b)
# end

# Pm = P_matrix(H_1, H_2)
# P_inv = P_inv_matrix(H_1, H_2)

# zeta_tilde = zeros(Float64, M+2, P+2, 2)
# psi_tilde = zeros(Float64, M+2, P+2, 2)

# for i in 1:2
#     zeta_tilde[:,:,i] = Pm[i,1]*zeta[:,:,1,1] + Pm[i,2]*zeta[:,:,2,1]
#     psi_tilde[:,:,i] = Pm[i,1]*psi[:,:,1,1] + Pm[i,2]*psi[:,:,2,1]
# end

In [6]:
function main()
    MINUTES = 60
    DAY = 60*60*24
    KM = 1000
    YEAR = 60*60*24*365

    H_1 = 1.0*KM
    H_2 = 2.0*KM
    beta = 2*10^-11
    Lx = 4000.0*KM # 4000 km
    Ly = 2000.0*KM # 2000 km
    dt = 30.0*MINUTES # 30 minutes
    T = 0.5*YEAR
    U = 1.0 # Forcing term of top level.
    M = 128
    dx = Lx / M
    P = Int(Ly / dx)
    visc = 100.0 # Viscosity, 100m^2s^-1
    r = 10^-7 # bottom friction scaler.
    R_d = 40.0*KM # Deformation radious, ~40km.
    
    simulation_name = "test7"
    file_name = "../../data/$simulation_name.jld"

    # M+2 and P+2 for ghost cells on each side.
    zeta = zeros(M+2, P+2, 2, 3)
    psi = zeros(M+2, P+2, 2, 3)

    # Initialise the model.
    for z in 1:2
        a = rand(Float64, (M+2, P+2))
        zeta[:,:,z,1] = update_ghost_cells(a)
        b = rand(Float64, (M+2, P+2))
        psi[:,:,z,1] = update_ghost_cells(b)
    end    

    sample_interval = 1.0*DAY
    sample_timestep = floor(Int, sample_interval / dt)

    metadata = Dict("dt" => dt,
        "T" => T,
        "sample_interval" => sample_interval,
        "sample_timestep" => sample_timestep
    )

    # Create the file and write the initial conditions.
    f = jldopen(file_name, "w") do file
        write(file, "zeta_0", zeta[:,:,:,1])
        write(file, "psi_0", psi[:,:,:,1])
        write(file, "metadata", metadata)
    end 

    # Create the model.
    model = BaroclinicModel(H_1, H_2, beta, Lx, Ly, dt, T, U, M, P, dx, visc, r, R_d)

    total_steps = floor(Int, T / dt)

    println("M = $M, P = $P")
    println("Timestep: $dt")
    println("T: $T")
    println("Total steps: $total_steps")

    for (timestep, time) in enumerate(1:dt:T)
        zeta = evolve_zeta_top(model, zeta, psi, timestep)
        zeta = evolve_zeta_bottom(model, zeta, psi, timestep)
        psi = evolve_psi(model, zeta, psi)

        if timestep % sample_timestep == 0
            t = timestep * time

            println("Timestep: $timestep, time: $t, | ", zeta[10,10,1,1])
            f = jldopen(file_name, "r+") do file
                write(file, "zeta_$timestep", zeta[:,:,:,1])
                write(file, "psi_$timestep", psi[:,:,:,1])
            end
        end
    end
end

main()

M = 128, P = 64
Timestep: 1800.0
T: 1.5768e7
Total steps: 8760
Timestep: 48, time: 4.060848e6, | 0.5065489783344718
Timestep: 96, time: 1.6416096e7, | 0.2570263136554847
Timestep: 144, time: 3.7065744e7, | 0.9780090654532486
Timestep: 192, time: 6.6009792e7, | 0.5252103292708794
Timestep: 240, time: 1.0324824e8, | 0.6387207303533964
Timestep: 288, time: 1.48781088e8, | 0.7113208059419306
Timestep: 336, time: 2.02608336e8, | 0.2571476561179255
Timestep: 384, time: 2.64729984e8, | 0.9939974554520224
Timestep: 432, time: 3.35146032e8, | 0.6545558944733004
Timestep: 480, time: 4.1385648e8, | 0.4854565568305661
Timestep: 528, time: 5.00861328e8, | 0.9206769488326116
Timestep: 576, time: 5.96160576e8, | 0.9495537653458228
Timestep: 624, time: 6.99754224e8, | 0.6935759163752815
Timestep: 672, time: 8.11642272e8, | 0.7476354823709894
Timestep: 720, time: 9.3182472e8, | 0.7689580865699855
Timestep: 768, time: 1.060301568e9, | 0.610610574830399
Timestep: 816, time: 1.197072816e9, | 0.61633972915